In [132]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

import sys
sys.path.append("..")

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import src.support.data_upload_support as dus
import time 

from unidecode import unidecode

### Cargar csv con estudio de keywords

In [348]:
keywords_df = pd.read_csv("../datos/keywords.csv")
keywords_df.columns = [columna.lower().replace(" ", "_").replace(".","_") for columna in keywords_df.columns]
keywords_df = keywords_df[["keyword", "currency", "avg__monthly_searches", "cambio_en_tres_meses", 
    "cambio_interanual", "competition", "competition_(indexed_value)", 
    "top_of_page_bid_(low_range)", "top_of_page_bid_(high_range)"]]
keywords_df.head()

,keyword,currency,avg__monthly_searches,cambio_en_tres_meses,cambio_interanual,competition,competition_(indexed_value),top_of_page_bid_(low_range),top_of_page_bid_(high_range)
0,tupper,EUR,18100.0,50%,23%,Alta,99.0,"0,11","0,39"
1,tupper cristal,EUR,2900.0,83%,22%,Alta,100.0,"0,14","0,37"
2,tupperware cristal,EUR,2900.0,83%,22%,Alta,100.0,"0,14","0,37"
3,tupper cristal ikea,EUR,1900.0,81%,21%,Alta,100.0,"0,09","0,43"
4,tupper ikea,EUR,1900.0,50%,0%,Alta,100.0,"0,05","0,52"


In [349]:
marcas_tuppers = ["tupperware", "ikea", "mercadona", "carrefour", "lidl", "luminarc", "lekue", "tatay", "tutete", "valira", "frigoverre", "sistema", "araven", "curver", "real madrid"]

def filter_out_brands(cadena, marcas):
    if not any([marca in cadena for marca in marcas]):
        return True

keywords_seleccionadas = list(filter(lambda x: filter_out_brands(x, marcas_tuppers),keywords_df["keyword"].to_list()))
keywords_seleccionadas
# filtrar por marca seleccionada
keywords_df = keywords_df[keywords_df["keyword"].isin(keywords_seleccionadas)]
keywords_df


,keyword,currency,avg__monthly_searches,cambio_en_tres_meses,cambio_interanual,competition,competition_(indexed_value),top_of_page_bid_(low_range),top_of_page_bid_(high_range)
0,tupper,EUR,18100.0,50%,23%,Alta,99.0,"0,11","0,39"
1,tupper cristal,EUR,2900.0,83%,22%,Alta,100.0,"0,14","0,37"
6,táper de cristal,EUR,1600.0,46%,19%,Alta,100.0,"0,10","0,30"
8,tupper de cristal,EUR,1000.0,82%,23%,Alta,100.0,"0,13","0,31"
10,tupper microondas,EUR,590.0,51%,-18%,Alta,100.0,"0,08","0,20"
...,...,...,...,...,...,...,...,...,...
1128,tupper keka amarilla,EUR,NaN,--,--,Desconocida,NaN,NaN,NaN
1129,tupper keka amarillo,EUR,NaN,--,--,Desconocida,NaN,NaN,NaN
1130,tupper para lunch miniso,EUR,NaN,--,--,Desconocida,NaN,NaN,NaN
1131,tupper salsa 3,EUR,NaN,--,--,Desconocida,NaN,NaN,NaN


Aplicar criterio de 100 búsquedas mínimas al mes.

In [350]:
keywords_df = keywords_df[keywords_df["avg__monthly_searches"] >= 100]
keywords_df.sort_values(by="competition",ascending=False)

,keyword,currency,avg__monthly_searches,cambio_en_tres_meses,cambio_interanual,competition,competition_(indexed_value),top_of_page_bid_(low_range),top_of_page_bid_(high_range)
15,tupper para llevar comida al trabajo,EUR,480.0,83%,22%,Media,49.0,"0,26","0,61"
77,tupper que es,EUR,170.0,91%,50%,Baja,11.0,NaN,NaN
0,tupper,EUR,18100.0,50%,23%,Alta,99.0,"0,11","0,39"
104,tupper congelador,EUR,140.0,91%,0%,Alta,100.0,"0,08","0,19"
73,tupper pequeño,EUR,170.0,191%,23%,Alta,100.0,"0,07","0,18"
...,...,...,...,...,...,...,...,...,...
57,tupper infantil compartimentos,EUR,210.0,321%,0%,Alta,100.0,"0,08","0,37"
58,tupper grande,EUR,210.0,0%,0%,Alta,100.0,"0,07","0,31"
59,tupper calienta comida,EUR,210.0,271%,0%,Alta,100.0,"0,08","0,77"
62,tupper cristal amazon,EUR,210.0,86%,24%,Alta,100.0,"0,07","0,13"


Coincide que la de competencia baja no contiene keywords que se relacionen con alta probabilidad de compra, así que se elimina.

In [158]:
keywords_df = keywords_df[keywords_df["keyword"]!="tupper que es"]

keywords_df["keyword"] = keywords_df["keyword"].apply(lambda x: unidecode(x))
keywords_df.sort_values(by="avg__monthly_searches",ascending=False)

,keyword,currency,avg__monthly_searches,cambio_en_tres_meses,cambio_interanual,competition,competition_(indexed_value),top_of_page_bid_(low_range),top_of_page_bid_(high_range),average_bid,rank_avg_monthly_searches,rank_cambio_en_tres_meses,rank_average_bid,score
0,tupper,EUR,18100.0,0.50,0.23,Alta,99.0,0.11,0.39,0.250,57,13,15,104.666667
1,tupper cristal,EUR,2900.0,0.83,0.22,Alta,100.0,0.14,0.37,0.255,56,23,13,107.333333
6,taper de cristal,EUR,1600.0,0.46,0.19,Alta,100.0,0.10,0.30,0.200,55,7,34,123.333333
8,tupper de cristal,EUR,1000.0,0.82,0.23,Alta,100.0,0.13,0.31,0.220,54,21,24,118.000000
10,tupper microondas,EUR,590.0,0.51,-0.18,Alta,100.0,0.08,0.20,0.140,53,14,47,142.666667
11,tupper infantil,EUR,480.0,1.71,-0.19,Alta,100.0,0.08,0.36,0.220,52,45,24,131.333333
12,tupper acero inoxidable,EUR,480.0,0.69,1.08,Alta,100.0,0.09,0.36,0.225,52,20,22,112.000000
13,tuppers hermeticos,EUR,480.0,0.85,0.00,Alta,100.0,0.09,0.33,0.210,52,27,26,122.000000
15,tupper para llevar comida al trabajo,EUR,480.0,0.83,0.22,Media,49.0,0.26,0.61,0.435,52,23,1,86.000000
16,tuppers acero inoxidable,EUR,480.0,0.69,1.08,Alta,100.0,0.09,0.36,0.225,52,20,22,112.000000


Limpiar cambio en tres meses e inter anual para eliminar simbolo de porcentajes. Formatear columnas de bid y encontrar el valor medio.

In [137]:
keywords_df[["cambio_en_tres_meses","cambio_interanual"]] = keywords_df[["cambio_en_tres_meses","cambio_interanual"]].map(lambda x: int(x.replace("%",""))/100)

keywords_df[["top_of_page_bid_(low_range)","top_of_page_bid_(high_range)"]] = keywords_df[["top_of_page_bid_(low_range)","top_of_page_bid_(high_range)"]].map(lambda x: x.replace(",",".")).astype(float)
keywords_df["average_bid"] = (keywords_df["top_of_page_bid_(low_range)"] + keywords_df["top_of_page_bid_(high_range)"]) / 2

In [143]:
from scipy.stats import rankdata

In [144]:
keywords_df = keywords_df[keywords_df["cambio_en_tres_meses"] > 0]
keywords_df.sort_values("average_bid").head()

,keyword,currency,avg__monthly_searches,cambio_en_tres_meses,cambio_interanual,competition,competition_(indexed_value),top_of_page_bid_(low_range),top_of_page_bid_(high_range),average_bid
100,fiambrera tupper,EUR,140.0,1.33,0.00,Alta,100.0,0.06,0.09,0.075
62,tupper cristal amazon,EUR,210.0,0.86,0.24,Alta,100.0,0.07,0.13,0.100
122,tupper bluey,EUR,110.0,1.89,0.00,Alta,100.0,0.09,0.12,0.105
23,tupper amazon,EUR,390.0,0.84,0.23,Alta,100.0,0.07,0.17,0.120
94,amazon tupper cristal,EUR,140.0,1.33,0.00,Alta,100.0,0.07,0.17,0.120


In [150]:
# Clasificar con rankdata: mayor valor = mejor clasificación
keywords_df['rank_avg_monthly_searches'] = rankdata(keywords_df['avg__monthly_searches'], method='max')
keywords_df['rank_cambio_en_tres_meses'] = rankdata(keywords_df['cambio_en_tres_meses'], method='max')

# Clasificar 'average_bid' para minimizar (menor valor = mejor clasificación)
keywords_df['rank_average_bid'] = rankdata(-keywords_df['average_bid'], method='max')

# Calcular la puntuación combinada promediando los rangos
keywords_df['score'] = (keywords_df['rank_avg_monthly_searches']*4 + keywords_df['rank_cambio_en_tres_meses']*2 + keywords_df['rank_average_bid']*4) / 3

# Ordenar por puntuación de mayor a menor
data = keywords_df.sort_values(by='score', ascending=False)

In [152]:
data.head(10)

,keyword,currency,avg__monthly_searches,cambio_en_tres_meses,cambio_interanual,competition,competition_(indexed_value),top_of_page_bid_(low_range),top_of_page_bid_(high_range),average_bid,rank_avg_monthly_searches,rank_cambio_en_tres_meses,rank_average_bid,score
23,tupper amazon,EUR,390.0,0.84,0.23,Alta,100.0,0.07,0.17,0.120,47,24,54,150.666667
10,tupper microondas,EUR,590.0,0.51,-0.18,Alta,100.0,0.08,0.20,0.140,53,14,47,142.666667
62,tupper cristal amazon,EUR,210.0,0.86,0.24,Alta,100.0,0.07,0.13,0.100,32,32,56,138.666667
73,tupper pequeno,EUR,170.0,1.91,0.23,Alta,100.0,0.07,0.18,0.125,26,53,51,138.000000
27,tupper para microondas,EUR,320.0,1.29,0.23,Alta,100.0,0.09,0.29,0.190,42,41,37,132.666667
100,fiambrera tupper,EUR,140.0,1.33,0.00,Alta,100.0,0.06,0.09,0.075,20,44,57,132.000000
11,tupper infantil,EUR,480.0,1.71,-0.19,Alta,100.0,0.08,0.36,0.220,52,45,24,131.333333
94,amazon tupper cristal,EUR,140.0,1.33,0.00,Alta,100.0,0.07,0.17,0.120,20,44,54,128.000000
6,taper de cristal,EUR,1600.0,0.46,0.19,Alta,100.0,0.10,0.30,0.200,55,7,34,123.333333
13,tuppers hermeticos,EUR,480.0,0.85,0.00,Alta,100.0,0.09,0.33,0.210,52,27,26,122.000000


### Escrapear resultados Google

In [362]:
driver = webdriver.Chrome()  # or specify path like: webdriver.Chrome(executable_path='/path/to/chromedriver')

driver.get("https://www.google.com/")

try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "L2AGLb"))).click()
except:
    pass

keyword_list = keywords_df.sort_values(by="avg__monthly_searches",ascending=False)["keyword"].to_list()
sopas = []
for keyword in keyword_list:
    keyword = keyword.replace(" ", "+")

    driver.get(f"https://www.google.com/search?q={keyword}")
    time.sleep(0.3)

    driver.execute_script("window.scrollTo(0, 1000);")

    time.sleep(0.3)

    page_source = driver.page_source
    sopa = BeautifulSoup(page_source, "html.parser")
    sopas.append(sopa)



In [368]:
idx = -1
keyword = keyword_list[idx].replace(" ","%20")
print(keyword)
sopas[idx].find("div",{"data-async-context":f"query:{keyword}"}).findChildren("h3")

for child in sopas[idx].find("div",{"data-async-context":f"query:{keyword}"}).findChildren("h3"):
    print(child)
    print(child.find_next_sibling().find("span",{"class":"VuuXrf"}).text)
    # print(child.find_parent("a")["href"])


tupper%20refrigerado
<h3 class="LC20lb MBeuO DKV0Md">Nevera Tupper</h3>
Amazon.es
<h3 class="LC20lb MBeuO DKV0Md">Tupper Hermético con Placa Refrigerante y Cajita ...</h3>
Las Mejores Botellas
<h3 class="LC20lb MBeuO DKV0Md">Tuppers y Fiambreras Térmicas | Online</h3>
Decathlon
<h3 class="LC20lb MBeuO DKV0Md">¿Cuánto duran cada uno de los alimentos en la nevera?</h3>
Cadena Dial
<h3 class="LC20lb MBeuO DKV0Md">¿Se pueden congelar los táper de cristal o de plástico? - 20Minutos</h3>
20Minutos
<h3 class="LC20lb MBeuO DKV0Md">¿Los tupper con comida caliente se estropean si los metes en la nevera?</h3>
The Objective
<h3 class="LC20lb MBeuO DKV0Md">ALIMENTOS | Thermos</h3>
Thermos
<h3 class="LC20lb MBeuO DKV0Md">Frigorifico I Tupperware</h3>
Tupperware España
<h3 class="LC20lb MBeuO DKV0Md">BOLSA NEVERA + TUPPER BZ5123</h3>
Benzi
<h3 class="LC20lb MBeuO DKV0Md">Porta alimentos | Menaje de cocina | Hogar</h3>
El Corte Inglés
<h3 class="LC20lb MBeuO DKV0Md">Tienda online de neveras y tuppers<

In [369]:
sem_shopping = []
top3_seo_url = []
top3_seo_name = []
top3_seo_h3 = []
for keyword, sopa in zip(keyword_list,sopas):
    keyword = keyword.replace(" ","%20")
    try:
        elementos = sopa.find("div",{"class":"top-pla-group-inner"}).findChildren("div",recursive=False)
        elementos = elementos[:min(3,len(elementos))]
        elementos_url = [elemento.findAll("a")[1]["href"] for elemento in elementos]
        sem_shopping.append(elementos_url)

    except:

        sem_shopping.append(np.nan)

    try:

        elementos_h3 = sopa.find("div",{"data-async-context":f"query:{keyword}"}).findChildren("h3")
        elementos_h3_text = [elemento.text for elemento in elementos_h3]
        elementos_name = [elemento.find_next_sibling().find("span",{"class":"VuuXrf"}).text for elemento in elementos_h3]
        elementos_url = [elemento.find_parent("a")["href"] for elemento in elementos_h3 if elemento.find_parent("a")]

        elementos_h3 = elementos_h3[:min(3,len(elementos_h3))]
        elementos_name = elementos_name[:min(3,len(elementos_name))]
        elementos_url = elementos_url[:min(3,len(elementos_url))]
        top3_seo_url.append(elementos_url)
        top3_seo_name.append(elementos_name)
        top3_seo_h3.append(elementos_h3_text)

    except:
        top3_seo_url.append(np.nan)
        top3_seo_name.append(np.nan)
        top3_seo_h3.append(np.nan)

    # try:
    #     elementos = sopa.findAll("g-inner-card")
    #     seo_shopping.append(elementos[:min(3,len(elementos))])
    #     print("Exito")
    # except:
    #     print("Except SEO shopping")
    #     seo_shopping.append(np.nan)


In [371]:
competencia = pd.DataFrame({
    "keyword": keyword_list,
    "urls_sem": sem_shopping,
    "urls_seo": top3_seo_url,
    "name_seo": top3_seo_name,
    "h3_seo": top3_seo_h3,
})
competencia

,keyword,urls_sem,urls_seo,name_seo,h3_seo
0,tupper,NaN,NaN,NaN,NaN
1,tupper cristal,[https://www.temu.com/es/kuiper/un9/shopping-g...,[https://www.amazon.es/tupper-cristal/s?k=tupp...,"[Amazon.es, Compra Online | Lidl, IKEA]","[Tupper Cristal, Tuppers para comida: de crist..."
2,táper de cristal,[https://www.temu.com/es/kuiper/un9/shopping-g...,NaN,NaN,NaN
3,tupper de cristal,[https://www.amazon.es/KICHLY-pl%C3%A1stico-al...,[https://www.amazon.es/tupper-cristal/s?k=tupp...,"[Amazon.es, Compra Online | Lidl, Ohgar]","[Tupper Cristal, Tuppers para comida: de crist..."
4,tupper microondas,[https://www.amazon.es/Quid-Recipiente-Rectang...,[https://www.tupperware.es/shop/es/microondas....,"[Tupperware España, La Tercera, SPG-Pack]","[Microondas I Tupperware, Cómo saber qué recip..."
...,...,...,...,...,...
59,tupper bluey,NaN,[https://www.amazon.es/Fiambrera-rectangular-a...,"[Amazon.es, Todojuguete, Carrefour]",[Fiambrera rectangular para el almuerzo infant...
60,tupper para conservar fruta cortada,NaN,[https://defiestaencasa.es/shop/reutilizables/...,"[DeFiestaEnCasa, Comunidad de Madrid |, Infobae]","[Caja para fruta tupperware naranja, 1 unidad,..."
61,tupper para tortilla,NaN,[https://www.amazon.es/tupper-tortilla/s?k=tup...,"[Amazon.es, Tupperware España, ALI Hogar]","[Tupper Tortilla, Tortilla Maker, Fiambrera pa..."
62,tupper para tortillas,NaN,[https://www.amazon.es/tupper-tortilla/s?k=tup...,"[Amazon.es, Tupperware España, Pochteca Papel ...","[Tupper Tortilla, Tortilla Maker, Ventajas del..."


In [372]:
competencia_sem = competencia[["keyword","urls_sem"]].explode("urls_sem").reset_index(drop=True)
competencia_seo_url = competencia[["keyword","urls_seo"]].explode("urls_seo").reset_index(drop=True)
competencia_seo_name = competencia[["name_seo"]].explode("name_seo").reset_index(drop=True)
competencia_seo_h3 = competencia[["h3_seo"]].explode("h3_seo").reset_index(drop=True)

competencia_total = pd.DataFrame()
for competencia in [competencia_seo_url,competencia_seo_name,competencia_seo_h3]:
    competencia_total = pd.concat([competencia_total,competencia],axis=1)
    

In [373]:
competencia_total.head()

,keyword,urls_seo,name_seo,h3_seo
0,tupper,NaN,NaN,NaN
1,tupper cristal,https://www.amazon.es/tupper-cristal/s?k=tuppe...,Amazon.es,Tupper Cristal
2,tupper cristal,https://www.lidl.es/es/tuppers/c6257?srsltid=A...,Compra Online | Lidl,"Tuppers para comida: de cristal, al vacío y más"
3,tupper cristal,https://www.ikea.com/es/es/p/ikea-365-bote-con...,IKEA,"IKEA 365+ bote con tapa, rectangular vidrio ..."
4,táper de cristal,NaN,NaN,Tapers cristal y plástico | Tuppers herméticos


In [375]:
keyword_data_competencia = competencia_total.merge(data, on="keyword")
keyword_data_competencia.to_csv("../datos/keywords_data_competencia.csv")

In [359]:
keywords_competencia =  competencia_total.merge(keywords_df, on="keyword")
keywords_competencia.to_csv("../datos/keywords_competencia.csv")

In [361]:
keywords_competencia_sem =  competencia_sem.merge(keywords_df, on="keyword")
keywords_competencia_sem.to_csv("../datos/keywords_competencia_sem.csv")

In [367]:
dus.upload_to_drive("../datos/keywords_competencia.csv","1vfttgGDHa3-eCMEN79QAue6aI3Gc1vvp")

RefreshError: ('invalid_grant: Invalid grant: account not found', {'error': 'invalid_grant', 'error_description': 'Invalid grant: account not found'})